<a href="https://colab.research.google.com/github/Majumota/ConstruccionDB_12FEB/blob/main/Construcci%C3%B3n_DB_Taller_BI_Majumota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Certainly! Designing a database schema for an online merch store requires understanding the entities involved and their relationships. Here's a basic schema for such a store:


*We'll use SQLite, a lightweight database that's integrated with Python, to create this online merch store database.*

Here's a Python script to set up the described database schema:

In [30]:
import sqlite3

**Creación de base de datos**

In [31]:
def create_carmax_v2_db():
    # Connect to SQLite database (it will create 'carmax_v2.db' if it doesn't exist)
    connection = sqlite3.connect('carmax_v2.db')
    cursor = connection.cursor()

       # Product table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Product (
        idProduct INTEGER PRIMARY KEY,
        nameProduct VARCHAR(255) NOT NULL,
        description TEXT NOT NULL,
        price DECIMAL(10,2) NOT NULL,
        stock INTEGER NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # Customer table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Customer (
        idCustomer INTEGER PRIMARY KEY,
        nameCustomer VARCHAR(255) NOT NULL,
        email TEXT NOT NULL,
        phone INTEGER NOT NULL UNIQUE,
        address TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # Sales table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Sales (
        idSales INTEGER PRIMARY KEY,
        idCustomer INTEGER NOT NULL,
        idProduct INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        total_price DECIMAL(10,2) NOT NULL,
        sale_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (idCustomer) REFERENCES Customer(idCustomer),
        FOREIGN KEY (idProduct) REFERENCES Product(idProduct)
    )
    ''')

    # Commit changes and close connection
    connection.commit()
    connection.close()

# Execute the function to create the database and tables
create_carmax_v2_db()


In [32]:
print (create_carmax_v2_db)

<function create_carmax_v2_db at 0x7893bb267ec0>


An Entity-Relationship (ER) model represents the logical structure of a database. The ER diagram showcases entities, their attributes, and the relationships between them.

Here's an ER diagram based on the database schema for the online merch store:

In [33]:
import sqlite3

def print_database():
    # Connect to the SQLite database
    connection = sqlite3.connect('carmax_v2.db')
    cursor = connection.cursor()

    # List of tables in our schema
    tables = ['Product', 'Customer', 'Sales']

    # Iterate over each table and print its contents
    for table in tables:
        print(f"===== {table} =====")

        # Fetch all rows from the table
        cursor.execute(f"SELECT * FROM {table}")
        rows = cursor.fetchall()

        # Fetch column names for the table
        cursor.execute(f"PRAGMA table_info({table})")
        columns = [column[1] for column in cursor.fetchall()]

        # Print column names
        print(", ".join(columns))

        # Print rows
        for row in rows:
            print(row)

        # Add a newline for separation
        print("\n")

    # Close the database connection
    connection.close()

# Execute the function to print the database contents
print_database()


===== Product =====
idProduct, nameProduct, description, price, stock, created_at
(1, 'Aceite de Motor', 'Aceite sintético de alta calidad para motor.', 72.32, 0, '2025-02-26 21:37:04')
(2, 'Filtro de Aire', 'Filtro de aire para vehículos livianos y pesados.', 153.82, 0, '2025-02-26 21:37:04')
(3, 'Batería 12V', 'Batería de alto rendimiento para autos.', 375.89, 0, '2025-02-26 21:37:04')
(4, 'Pastillas de Freno', 'Juego de pastillas de freno delanteras.', 148.07, 0, '2025-02-26 21:37:04')
(5, 'Llantas Radiales', 'Llantas de alta resistencia para carretera.', 305.88, 0, '2025-02-26 21:37:04')
(6, 'Anticongelante', 'Líquido refrigerante para motores.', 341.97, 0, '2025-02-26 21:37:04')
(7, 'Amortiguadores', 'Amortiguadores para suspensión delantera.', 222.2, 0, '2025-02-26 21:37:04')
(8, 'Bujías', 'Juego de bujías de iridio para encendido eficiente.', 208.19, 0, '2025-02-26 21:37:04')
(9, 'Aceite de Transmisión', 'Lubricante para transmisión automática.', 434.02, 0, '2025-02-26 21:37:04'

**Importar datos random con faker**

In [35]:
!pip install faker

**Poblamiento de las tablas Product, Customer, Sale con datos random**

In [36]:
import sqlite3
import random
from faker import Faker

# Inicializar Faker en español para Colombia
fake = Faker("es_CO")

In [37]:
# Lista de productos del sector automotriz
automotive_products = [
    ("Aceite de Motor", "Aceite sintético de alta calidad para motor."),
    ("Filtro de Aire", "Filtro de aire para vehículos livianos y pesados."),
    ("Batería 12V", "Batería de alto rendimiento para autos."),
    ("Pastillas de Freno", "Juego de pastillas de freno delanteras."),
    ("Llantas Radiales", "Llantas de alta resistencia para carretera."),
    ("Anticongelante", "Líquido refrigerante para motores."),
    ("Amortiguadores", "Amortiguadores para suspensión delantera."),
    ("Bujías", "Juego de bujías de iridio para encendido eficiente."),
    ("Aceite de Transmisión", "Lubricante para transmisión automática."),
    ("Kit de Correa de Distribución", "Juego de correa y tensores para motor.")
]

# Conectar a la base de datos
connection = sqlite3.connect('carmax_v2.db')
cursor = connection.cursor()

# Limpiar las tablas antes de poblar (opcional, si deseas regenerar datos)
cursor.execute("DELETE FROM Sales")
cursor.execute("DELETE FROM Customer")
cursor.execute("DELETE FROM Product")
connection.commit()

# Poblar la tabla Product con productos automotrices
products = []
for name, description in automotive_products:
    price = round(random.uniform(20, 500), 2)  # Precio entre 20 y 500
    stock = random.randint(5, 50)  # Stock entre 5 y 50
    cursor.execute("INSERT INTO Product (nameProduct, description, price, stock) VALUES (?, ?, ?, ?)",
                   (name, description, price, stock))
    products.append(cursor.lastrowid)  # Guardar ID del producto insertado

# Poblar la tabla Customer con clientes en Colombia
customers = []
for _ in range(500):
    name = fake.name()
    email = fake.unique.email()
    phone = fake.unique.numerify(text="3#########")  # Simula un número colombiano
    address = fake.street_address() + ", " + fake.city() + ", " + fake.department()
    cursor.execute("INSERT INTO Customer (nameCustomer, email, phone, address) VALUES (?, ?, ?, ?)",
                   (name, email, phone, address))
    customers.append(cursor.lastrowid)  # Guardar ID del cliente insertado

# Poblar la tabla Sales con ventas realistas
for _ in range(25000):
    idCustomer = random.choice(customers)
    idProduct = random.choice(products)

    # Obtener stock del producto
    cursor.execute("SELECT stock, price FROM Product WHERE idProduct = ?", (idProduct,))
    stock, price = cursor.fetchone()

    if stock > 0:
        quantity = random.randint(1, min(stock, 5))  # Vender entre 1 y stock disponible
        total_price = round(quantity * price, 2)

        cursor.execute("INSERT INTO Sales (idCustomer, idProduct, quantity, total_price) VALUES (?, ?, ?, ?)",
                       (idCustomer, idProduct, quantity, total_price))

        # Reducir el stock del producto vendido
        cursor.execute("UPDATE Product SET stock = stock - ? WHERE idProduct = ?", (quantity, idProduct))

# Guardar los cambios y cerrar la conexión
connection.commit()
connection.close()

print("Base de datos poblada correctamente con datos automotrices y ubicaciones en Colombia.")

Base de datos poblada correctamente con datos automotrices y ubicaciones en Colombia.


**Imprimir tablas con datos**

In [38]:
!pip install pandas

In [39]:
import sqlite3
import pandas as pd

In [40]:
def print_database():
    # Conectar a la base de datos
    connection = sqlite3.connect('carmax_v2.db')

    # Listado de tablas
    tables = ['Product', 'Customer', 'Sales']

    # Iterar sobre cada tabla y mostrarla con pandas
    for table in tables:
        print(f"\n===== {table} =====\n")

        # Leer los datos de la tabla con pandas
        df = pd.read_sql_query(f"SELECT * FROM {table}", connection)

        # Mostrar la tabla en formato DataFrame
        display(df)

    # Cerrar la conexión
    connection.close()

# Ejecutar la función para imprimir la base de datos
print_database()



===== Product =====



,idProduct,nameProduct,description,price,stock,created_at
0,1,Aceite de Motor,Aceite sintético de alta calidad para motor.,467.46,0,2025-02-26 21:37:57
1,2,Filtro de Aire,Filtro de aire para vehículos livianos y pesados.,198.08,0,2025-02-26 21:37:57
2,3,Batería 12V,Batería de alto rendimiento para autos.,194.68,0,2025-02-26 21:37:57
3,4,Pastillas de Freno,Juego de pastillas de freno delanteras.,319.86,0,2025-02-26 21:37:57
4,5,Llantas Radiales,Llantas de alta resistencia para carretera.,387.49,0,2025-02-26 21:37:57
5,6,Anticongelante,Líquido refrigerante para motores.,482.10,0,2025-02-26 21:37:57
6,7,Amortiguadores,Amortiguadores para suspensión delantera.,313.29,0,2025-02-26 21:37:57
7,8,Bujías,Juego de bujías de iridio para encendido efici...,183.07,0,2025-02-26 21:37:57
8,9,Aceite de Transmisión,Lubricante para transmisión automática.,151.22,0,2025-02-26 21:37:57
9,10,Kit de Correa de Distribución,Juego de correa y tensores para motor.,85.79,0,2025-02-26 21:37:57



===== Customer =====



,idCustomer,nameCustomer,email,phone,address,created_at
0,1,Yury Lucila Jaramillo,isabel07@example.net,3991715877,"Cr. 98 # 37-8, Argelia, Bolívar",2025-02-26 21:37:57
1,2,Juan Estrada,alfredo07@example.org,3229782041,"Carrera 102I # 8-39 Sur, Villagómez, Caldas",2025-02-26 21:37:57
2,3,Gregorio Gómez Vera,felipe80@example.com,3038314882,"Calle 5ª # 26-9 Sur, Onzaga, Tolima",2025-02-26 21:37:57
3,4,Edgar Ángel Valencia Ríos,robertfonseca@example.net,3265459259,"Calle 170 # 99-7 Este, Guapotá, Risaralda",2025-02-26 21:37:57
4,5,Marina Jazmín Martínez,liceth62@example.com,3338797552,"Calle 6ª # 59-4 Sur, Colombia, Nariño",2025-02-26 21:37:57
...,...,...,...,...,...,...
495,496,Guillermo Gutiérrez Fernández,canopedro@example.net,3964351637,"Cl. 61 # 75-70 Este, Filadelfia, Putumayo",2025-02-26 21:37:58
496,497,Inés Londoño,enrique18@example.net,3559372281,"Calle 36 # 91-1 Sur, San Rafael, Santander",2025-02-26 21:37:58
497,498,Luis Jairo García,sofiagarcia@example.net,3862777393,"Cr. 53 # 71-67, Barbacoas, Vichada",2025-02-26 21:37:58
498,499,Juan Villamil,aura69@example.org,3869332747,"Av. carrera 112 # 7-13, Consacá, Córdoba",2025-02-26 21:37:58



===== Sales =====



,idSales,idCustomer,idProduct,quantity,total_price,sale_date
0,1,179,4,1,319.86,2025-02-26 21:37:58
1,2,303,6,2,964.20,2025-02-26 21:37:58
2,3,244,7,1,313.29,2025-02-26 21:37:58
3,4,258,4,1,319.86,2025-02-26 21:37:58
4,5,223,4,3,959.58,2025-02-26 21:37:58
...,...,...,...,...,...,...
119,120,105,8,1,183.07,2025-02-26 21:37:58
120,121,350,3,3,584.04,2025-02-26 21:37:58
121,122,72,3,2,389.36,2025-02-26 21:37:58
122,123,448,8,2,366.14,2025-02-26 21:37:58


In [ ]:
#Prompt: Ahora necesito descargar en un path en google drive esta db en sqlite
from google.colab import drive
drive.mount('/content/drive')

import shutil

source_path = '/content/carmax_v2.db'
destination_path = '/content/drive/MyDrive/carmax_v2.db'

shutil.copy(source_path, destination_path)

print(f"Database file copied to: {destination_path}")

Mounted at /content/drive
Database file copied to: /content/drive/MyDrive/carmax_v2.db
